In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
def clean_census_frame(file_path):
    '''
    input) file_path
        >> path to csv
    '''
    # load data
    df = pd.read_csv( file_path , low_memory=False )

    # and copy
    _df = df.copy()

    # reset column names to current 0th row values
    _df.columns = _df.iloc[0]
    # new 2000 dataframe without row where values are from
    _df = _df[1:]
    # reset index
    clean_df = _df.reset_index()
    
    return clean_df

In [3]:
# 2011
y2k11 = clean_census_frame('../data/acs/aff_download/ACS_11_5YR_DP05_with_ann.csv')
# 2012
y2k12 = clean_census_frame('../data/acs/aff_download/ACS_12_5YR_DP05_with_ann.csv')
#2013
y2k13 = clean_census_frame('../data/acs/aff_download/ACS_13_5YR_DP05_with_ann.csv')
# 2014
y2k14 = clean_census_frame('../data/acs/aff_download/ACS_14_5YR_DP05_with_ann.csv')
# 2015
y2k15 = clean_census_frame('../data/acs/aff_download/ACS_15_5YR_DP05_with_ann.csv')
# 2016
y2k16 = clean_census_frame('../data/acs/aff_download/ACS_16_5YR_DP05_with_ann.csv')
# 2017
y2k17 = clean_census_frame('../data/acs/aff_download/ACS_17_5YR_DP05_with_ann.csv')

In [4]:
'''MAKE IT FASTER >> done to speed up modeling process'''

# 2011
y2k11 = y2k11.head(1000)
# 2012
y2k12 = y2k12.head(1000)
#2013
y2k13 = y2k13.head(1000)
# 2014
y2k14 = y2k14.head(1000)
# 2015
y2k15 = y2k15.head(1000)
# 2016
y2k16 = y2k16.head(1000)
# 2017
y2k17 = y2k17.head(1000)

In [5]:
y2k11

,index,Id,Id2,Geography,Estimate; SEX AND AGE - Total population,Margin of Error; SEX AND AGE - Total population,Percent; SEX AND AGE - Total population,Percent Margin of Error; SEX AND AGE - Total population,Estimate; SEX AND AGE - Male,Margin of Error; SEX AND AGE - Male,...,Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races including Some other race,"Estimate; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent Margin of Error; HISPANIC OR LATINO AND RACE - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races",Estimate; HISPANIC OR LATINO AND RACE - Total housing units,Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units,Percent; HISPANIC OR LATINO AND RACE - Total housing units,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
0,1,8600000US00601,00601,ZCTA5 00601,18533,310,18533,(X),8971,208,...,0.0,0.2,0,92,0.0,0.2,6503,164,(X),(X)
1,2,8600000US00602,00602,ZCTA5 00602,41930,136,41930,(X),20568,84,...,0.0,0.1,228,125,0.5,0.3,16336,225,(X),(X)
2,3,8600000US00603,00603,ZCTA5 00603,54475,839,54475,(X),26588,459,...,0.0,0.1,250,130,0.5,0.2,23245,334,(X),(X)
3,4,8600000US00606,00606,ZCTA5 00606,6386,291,6386,(X),3085,186,...,0.0,0.5,0,92,0.0,0.5,2373,111,(X),(X)
4,5,8600000US00610,00610,ZCTA5 00610,29111,173,29111,(X),14162,86,...,0.0,0.1,0,92,0.0,0.1,11308,147,(X),(X)
5,6,8600000US00612,00612,ZCTA5 00612,70541,1436,70541,(X),33449,835,...,0.0,0.1,40,46,0.1,0.1,29575,452,(X),(X)
6,7,8600000US00616,00616,ZCTA5 00616,10617,866,10617,(X),5166,505,...,0.0,0.3,3,6,0.0,0.1,4466,264,(X),(X)
7,8,8600000US00617,00617,ZCTA5 00617,24458,261,24458,(X),11752,98,...,0.0,0.1,17,28,0.1,0.1,9627,192,(X),(X)
8,9,8600000US00622,00622,ZCTA5 00622,5419,854,5419,(X),2392,400,...,0.0,0.6,0,92,0.0,0.6,7042,351,(X),(X)
9,10,8600000US00623,00623,ZCTA5 00623,45314,854,45314,(X),21904,400,...,0.0,0.1,0,92,0.0,0.1,19997,390,(X),(X)


In [7]:
# gather column names
column_names = [_ for _ in y2k11[:0]]
column_names

['index',
 'Id',
 'Id2',
 'Geography',
 'Estimate; SEX AND AGE - Total population',
 'Margin of Error; SEX AND AGE - Total population',
 'Percent; SEX AND AGE - Total population',
 'Percent Margin of Error; SEX AND AGE - Total population',
 'Estimate; SEX AND AGE - Male',
 'Margin of Error; SEX AND AGE - Male',
 'Percent; SEX AND AGE - Male',
 'Percent Margin of Error; SEX AND AGE - Male',
 'Estimate; SEX AND AGE - Female',
 'Margin of Error; SEX AND AGE - Female',
 'Percent; SEX AND AGE - Female',
 'Percent Margin of Error; SEX AND AGE - Female',
 'Estimate; SEX AND AGE - Under 5 years',
 'Margin of Error; SEX AND AGE - Under 5 years',
 'Percent; SEX AND AGE - Under 5 years',
 'Percent Margin of Error; SEX AND AGE - Under 5 years',
 'Estimate; SEX AND AGE - 5 to 9 years',
 'Margin of Error; SEX AND AGE - 5 to 9 years',
 'Percent; SEX AND AGE - 5 to 9 years',
 'Percent Margin of Error; SEX AND AGE - 5 to 9 years',
 'Estimate; SEX AND AGE - 10 to 14 years',
 'Margin of Error; SEX AND AG

In [8]:
for _ in [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]:
    print(_.info(),'\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 2.5+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 2.5+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 2.5+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 328 entries, index to Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
dtypes: int64(1), object(327)
memory usage: 2.5+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeI

In [16]:
# determine columns in 2015, 2016 and 2017 which are not seen in 2011-2014 
eleven_columns = [name for name in y2k11.columns]
twelve_columns = [name for name in y2k12.columns]
thirteen_columns = [name for name in y2k12.columns]
fourteen_columns = [name for name in y2k12.columns]
fifteen_columns = [name for name in y2k15.columns]
sixteen_columns = [name for name in y2k16.columns]
seventeen_columns = [name for name in y2k17.columns]

In [25]:
'''make sure those with 340 columns all have the same columns'''

# count non coexist 
same_len_non_coexist = 0

# compare all to initial 
for name in eleven_columns: 
    if name not in twelve_columns:
        same_len_non_coexist+=1
        # raise Exception(f'FLAWED ASSUMPTION if name in y2k12.columns name = {name}')
    elif name not in thirteen_columns:
        same_len_non_coexist+=1
        #raise Exception(f'FLAWED ASSUMPTION if name in y2k13.columns name = {name}')
    elif name not in fourteen_columns:
        same_len_non_coexist+=1
        #raise Exception(f'FLAWED ASSUMPTION if name in y2k14.columns name = {name}')
if same_len_non_coexist > 0:
    raise Exception(f'FLAWED ASSUMPTION same_len_non_coexist ({same_len_non_coexist}) > 0')

In [43]:
'''see if those 340 column names are seen in 2015, 2016, and 2017'''

# count non coexist
non_coexist_2015 = 0

# compare to initial 
for _ in range(len(eleven_columns)): 
    if eleven_columns[_] not in seventeen_columns and eleven_columns[_] not in sixteen_columns and eleven_columns[_] not in fifteen_columns:
        # update count
        non_coexist_2015+=1
        # show the name and name at same index in 2015
        # print(f'2011\n{name}\n2015\n')
        for _ in fifteen_columns[_-5:_+5]:
            # print(_)
            pass
        # print('\n\n')
        # raise Exception(f'FLAWED ASSUMPTION if name in y2k15.columns name = {name}')
non_coexist_2015,len(eleven_columns)


(108, 328)

In [ ]:
#     elif name not in sixteen_columns:
#         raise Exception(f'FLAWED ASSUMPTION if name in y2k16.columns name = {name}')
#     elif name not in seventeen_columns:
#         raise Exception(f'FLAWED ASSUMPTION if name in y2k17.columns name = {name}')

In [ ]:
# make dataframe for each zip code, should contain row from each year
def frame_per_zip(dataframes):
    '''
    take dataframe for each year set is available
    find common zipcode for each year
    make dataframe for that zipcode
        containing each year's measurements
    note:
        dataframes must be same length
    '''
    # check length of dataframes is same
    for _ in range(len(dataframes)):
        # done to ensure each zip code has same represnetation 
        if len(dataframes[_-1]) != len(dataframes[_]):
            # stop if we have different lengths
            raise Exception(f'len(dataframe[{-_}]) != len(dataframe[{_}])')
        # also check that dataframes have same Id
        random_samples = [random.randint(1,int(len(dataframes[-_])/2)),
                          random.randint(1,int(len(dataframes[-_]))),
                          random.randint(int(len(dataframes[-_])/2),len(dataframes[-_]))]
        for sample in random_samples:
            # pull Id from df to compare
            if dataframes[_]['Id'][sample] != dataframes[_-2]['Id'][sample]:
                # stop if they don't match
                raise Exception(f"NON MATCHING Id\n{dataframes[_].Id[sample]}\nERROR\n{dataframes[_-2].Id[sample]}\n")
    
    # now we can get to work
    mini_dfs = []
    for i in range(len(dataframes[0])):
        mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=dataframes[0].columns)
        for _ in range(len(dataframes)):
            q = dataframes[_]
            print(q)
#             mini_df[_] = q.index(i)
        mini_dfs.append(mini_df)
    
    print(len(mini_df))
        
    return mini_dfs
        
    

In [ ]:
yrs = [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]
z = frame_per_zip(yrs)

In [ ]:
for year in [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]:
    print (year.columns)

In [ ]:
mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=['a','b','c','d','e'])
mini_df[1:2]

In [ ]:
# dataframes = [y2k11,y2k12,y2k13,y2k14,y2k15,y2k16,y2k17]
# mini_df = pd.DataFrame(index=['2011','2012','2013','2014','2015','2016','2017'], columns=dataframes[0].columns)
# mini_df.iloc[0] = 'lol'
# mini_df.iloc[0]
# mini_df

In [ ]:
'''# load 2000 data
y2k = pd.read_csv( a , low_memory=False )
# load 2010 data
y2k10 = pd.read_csv( b , low_memory=False )

# 2000 Census
b = y2k.copy()
# 2010 Census
o = y2k10.copy()

# reset 2000 columns to current 0th row values
b.columns = b.iloc[0]
# new 2000 dataframe without row where values are from
b = b[1:]
# reset index
b = b.reset_index()

# reset 2010 columns to current 0th row values
o.columns = o.iloc[0]
# new 2010 dataframe without row where values are from
o = o[1:]
# reset index
o = o.reset_index()

# identify zip codes from 2000 .Geography (last 5 chars of string)
zip_2000_codes = [q[-5:] for q in b.Geography]  # ValueError: invalid literal for int() with base 10: '006HH'
# identify zip codes from 2010 .Geography (last 5 chars of string)
zip_2010_codes = [q[-5:] for q in o.Geography]

# from 2000.Geography , instance is not seen in 2010.Geography  -- sample: zip_code = (2, 'c')
in_2000_but_not_2010_from_2000 = [zip_code for zip_code in enumerate(zip_2000_codes) if zip_code[1] not in zip_2010_codes]
# from 2010.Geography , instance is not seen in 2000.Geography  -- sample: zip_code[1] = 'c'
in_2010_but_not_2000_from_2010 = [zip_code for zip_code in enumerate(zip_2010_codes) if zip_code[1] not in zip_2000_codes]

# from 2000.Geography , instance is seen in 2010.Geography
in_2000_and_2010_from_2000 = [zip_code for zip_code in enumerate(zip_2000_codes) if zip_code[1] in zip_2010_codes]
# from 2010.Geography , instance is seen in 2000.Geography
in_2010_and_2000_from_2010 = [zip_code for zip_code in enumerate(zip_2010_codes) if zip_code[1] in zip_2000_codes]

# index of objects coexisting in 2000 and 2010
of_2000_indexes = [i for i,j in in_2000_and_2010_from_2000]
# index of objects coexisting in 2010 and 2000 
of_2010_indexes = [i for i,j in in_2010_and_2000_from_2010]
# ^note: these are different lists, if took j instead of i, then would be same list
if [j for i,j in in_2000_and_2010_from_2000] != [j for i,j in in_2010_and_2000_from_2010]:
    # like is seen here, j for j == True
    raise Exception(f'FLAWED ASSUMPTION , [j for i,j in 2000] != [j for i,j in 2010]\n'
                    f'len {len(in_2000_and_2010_from_2000)} {len(in_2010_and_2000_from_2010)}')
# however i for i == False
if of_2000_indexes == of_2010_indexes:
    # cheers
    raise Exception('FLAWED ASSUMPTION , of_2000_indexes != of_2010_indexes\n'
                    f'len y2k {len(of_2000_indexes)} 2k10 {len(of_2010_indexes)}')  

# thin 2000 to shared geo
b = b.iloc[of_2000_indexes]
# thin 2010 to shared geo
o = o.iloc[of_2010_indexes]'''
pass